In [53]:
import pandas as pd
import json
import os
import deepsig
from IPython.display import display

In [54]:
cols = ['dataset', 'method', 'fitness_rule', 'fitness', 'ACC', 'MCC', 'f1_score', 'avg_odds_diff', 'stat_par_diff', 'eq_opp_diff']

In [55]:
mlp_baseline_results = pd.read_csv('simple_mlp_results.csv')
mlp_baseline_results.replace({'simple_mlp_initializer': r'MLP'}, inplace=True)

mlp_standard_l2_results = pd.read_csv('mlp_standard_l2_results.csv')
mlp_standard_l2_results.replace({'mlp_standard_l2_initializer': r'MLP+L2'}, inplace=True)

mlp_preg_results = pd.read_csv('mlp_preg_results.csv')
mlp_preg_results.replace({'mlp_preg_initializer': r'MLP+RPR$_{\rho}$'}, inplace=True)

mlp_sreg_results = pd.read_csv('mlp_sreg_results.csv')
mlp_sreg_results.replace({'mlp_sreg_initializer': r'MLP+RPR$_{\rho_s}$'}, inplace=True)

mlp_kreg_results = pd.read_csv('mlp_kreg_results.csv')
mlp_kreg_results.replace({'mlp_kreg_initializer': r'MLP+RPR$_{\tau}$'}, inplace=True)

mlp_xi_reg_results = pd.read_csv('mlp_xi_reg_results.csv')
mlp_xi_reg_results.replace({'mlp_xi_reg_initializer': r'MLP+RPR$_{\xi}$'}, inplace=True)


results = pd.concat([mlp_baseline_results, mlp_standard_l2_results, mlp_preg_results, mlp_sreg_results, mlp_kreg_results, mlp_xi_reg_results], ignore_index=True)


In [56]:
results.replace({'adult_dataset_reader': 'Adult Income', 'compas_dataset_reader': 'Compas Recidivism', 'german_dataset_reader': 'German Credit', 'bank_dataset_reader': 'Bank Marketing'}, inplace=True)
results.rename(columns={'avg_odds_diff': 'Equalized Odds', 'stat_par_diff': 'Statistical Parity', 'eq_opp_diff': 'Equal Opportunity', 'MCC': 'Mathew Correlation', 'ACC': 'Accuracy'}, inplace=True)

In [57]:
fitness_rules_target_metrics = {
    'mcc_parity': {'performance': 'Mathew Correlation', 'fairness': 'Statistical Parity'},
    'mcc_opportunity': {'performance': 'Mathew Correlation', 'fairness': 'Equal Opportunity'},
    'mcc_odds': {'performance': 'Mathew Correlation', 'fairness': 'Equalized Odds'},
    'acc_parity': {'performance': 'Accuracy', 'fairness': 'Statistical Parity'},
    'acc_opportunity': {'performance': 'Accuracy', 'fairness': 'Equal Opportunity'},
    'acc_odds': {'performance': 'Accuracy', 'fairness': 'Equalized Odds'}
}

fitness_rules_target_metrics = {
    'mcc_parity': ('Mathew Correlation', 'Statistical Parity'),
    'mcc_opportunity': ('Mathew Correlation', 'Equal Opportunity'),
    'mcc_odds': ('Mathew Correlation', 'Equalized Odds'),
    'acc_parity': ('Accuracy', 'Statistical Parity'),
    'acc_opportunity': ('Accuracy', 'Equal Opportunity'),
    'acc_odds': ('Accuracy', 'Equalized Odds')
}
fitness_rules_abvr = {
    'mcc_parity': 'Max(MCC - Stat. Parity)',
    'mcc_opportunity': 'Max(MCC - Eq. Odds)',
    'mcc_odds': 'Max(MCC - Eq. Opp.)',
    'acc_parity': 'Max(Acc - Stat. Parity)',
    'acc_opportunity': 'Max(Acc - Eq. Odds)',
    'acc_odds': 'Max(Acc - Eq. Opp.)'
}

results['Performance'] = 0
results['Fairness'] = 0
results['Fitness Rule'] = ''
for fitness_rule, (performance_metric, fairness_metric) in fitness_rules_target_metrics.items():
    results.loc[results.fitness_rule == fitness_rule,'Performance'] = results.loc[results.fitness_rule == fitness_rule,performance_metric]
    results.loc[results.fitness_rule == fitness_rule,'Fairness'] = results.loc[results.fitness_rule == fitness_rule,fairness_metric]
    results.loc[results.fitness_rule == fitness_rule,'Fitness Rule Abvr'] = fitness_rules_abvr[fitness_rule]
    results.loc[results.fitness_rule == fitness_rule,'Fitness Rule'] = 'Max(%s - %s)' % fitness_rules_target_metrics[fitness_rule]

/tmp/ipykernel_6609/3340472993.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.58517004 0.58512823 0.5840247  0.5836371  0.57625737 0.5769683
 0.58075151 0.57811729 0.57147862 0.55847176 0.56402881 0.58191431
 0.58343253 0.57740222 0.51579307 0.30958237 0.37962947 0.55899189
 0.53558872 0.2598879  0.57717035 0.50086739 0.33275221 0.37929293
 0.47729109 0.29196869 0.29037704 0.52303609 0.27480633 0.38668781
 0.50626951 0.29960979 0.4042848  0.52119218 0.2936422  0.33522388
 0.49730721 0.25112662 0.35119067 0.510052   0.26843855 0.36081008
 0.52157495 0.30114722 0.42995147 0.48267704 0.29056691 0.23139881
 0.54223416 0.23653437 0.27041017 0.52121263 0.25759348 0.23912165
 0.5223892  0.27406652 0.22303564 0.54094974 0.27699558 0.30939251
 0.51731345 0.30339828 0.37825089 0.52113734 0.26323857 0.33963196
 0.5538225  0.27850228 0.30234984 0.51877126 0.29494101 0.2648939
 0.52742877 0.24492927 0.13407928 0.535303

In [58]:
datasets = ['Adult Income', 'Bank Marketing', 'Compas Recidivism','German Credit']
datasets

['Adult Income', 'Bank Marketing', 'Compas Recidivism', 'German Credit']

In [59]:
fitness_rules = ['mcc_parity', 'mcc_opportunity', 'mcc_odds', 'acc_parity', 'acc_opportunity', 'acc_odds']
fitness_rules

['mcc_parity',
 'mcc_opportunity',
 'mcc_odds',
 'acc_parity',
 'acc_opportunity',
 'acc_odds']

In [60]:
methods = [r'MLP', r'MLP+L2', r'MLP+RPR$_{\rho}$', r'MLP+RPR$_{\rho_s}$', r'MLP+RPR$_{\tau}$', r'MLP+RPR$_{\xi}$']
significances = []
grouped_results_list = []

In [61]:
path = 'mlp_multi_aso_data_list.json'

if os.path.exists(path):
    with open(path) as file:
        multi_aso_data_list = json.load(file)
else:    
    multi_aso_data_list = []
    for d in datasets:
        multi_aso_data = []
        for f in fitness_rules:
            methods_results = []
            for m in methods:
                r = results.loc[ (results['dataset'] == d) &
                                     (results['fitness_rule'] == f) &
                                     (results['method'] == m) ]\
                            .fitness.tolist()
                if len(r) == 0:
                    r = [-1]
                methods_results.append(r)
            min_eps = deepsig.multi_aso(methods_results, confidence_level=0.95)
            multi_aso_data_list.append({'fitness_rule': f, 'dataset': d, 'min_eps': min_eps.tolist()})
    with open(path, 'w') as file:
        json.dump(multi_aso_data_list, file)

aso_df_resume = []
for aso_result in sorted(multi_aso_data_list, key=lambda x: x['dataset']):
    fitness_rule = aso_result['fitness_rule']
    dataset = aso_result['dataset']

    aso_df = pd.DataFrame(aso_result['min_eps'], columns=methods)
    aso_df['method'] = methods
    aso_df['dataset'] = dataset
    aso_df['fitness_rule'] = fitness_rule
    aso_df_resume.append(aso_df)

print('Significance Testing')
significance = pd.concat(aso_df_resume)
significance.replace(fitness_rules_abvr, inplace=True)
significance = significance.set_index(['fitness_rule', 'dataset'])
significance = significance.sort_values(by=['fitness_rule', 'dataset'], ascending=[False, True])
significances.append(significance)

grouped_results = results\
    .groupby(['Fitness Rule Abvr', 'dataset', 'method'])\
    .agg({'fitness': ['mean', 'std', 'count'], 'Performance': ['mean', 'std'], 'Fairness': ['mean', 'std']})\
    .sort_values(by=['Fitness Rule Abvr', 'dataset', ('fitness','mean')], ascending=False)
grouped_results['formatted_fitness'] = grouped_results.apply(lambda row: f"${row[('fitness', 'mean')]:.3f} (\pm{row[('fitness', 'std')]:.2f})$", axis=1)
grouped_results['formatted_performance'] = grouped_results.apply(lambda row: f"${row[('Performance', 'mean')]:.3f} (\pm{row[('Performance', 'std')]:.2f})$", axis=1)
grouped_results['formatted_fairness'] = grouped_results.apply(lambda row: f"${row[('Fairness', 'mean')]:.3f} (\pm{row[('Fairness', 'std')]:.2f})$", axis=1)
grouped_results = grouped_results.sort_values(by=['Fitness Rule Abvr', 'dataset'])
grouped_results_list.append(grouped_results)

Model comparisons: 100%|█████████▉| 14985/15000 [00:40<00:00, 369.94it/s]


Significance Testing


In [62]:
grouped_results = results\
    .groupby(['fitness_rule', 'dataset', 'method'])\
    .agg({'fitness': ['mean', 'std', 'count'], 'Performance': ['mean', 'std'], 'Fairness': ['mean', 'std']})\
    .sort_values(by=['fitness_rule', 'dataset', ('fitness','mean')], ascending=[False, True, False])
grouped_results['formatted_fitness'] = grouped_results.apply(lambda row: f"${row[('fitness', 'mean')]:.3f} (\pm{row[('fitness', 'std')]:.2f})$", axis=1)
grouped_results['formatted_performance'] = grouped_results.apply(lambda row: f"${row[('Performance', 'mean')]:.3f} (\pm{row[('Performance', 'std')]:.2f})$", axis=1)
grouped_results['formatted_fairness'] = grouped_results.apply(lambda row: f"${row[('Fairness', 'mean')]:.3f} (\pm{row[('Fairness', 'std')]:.2f})$", axis=1)
grouped_results

fitness                  \
                                                   mean       std count   
fitness_rule dataset       method                                         
mcc_parity   Adult Income  MLP+RPR$_{\rho}$    0.393134  0.012324    18   
                           MLP+RPR$_{\xi}$     0.387579  0.010264    15   
                           MLP                 0.385655  0.010666    15   
                           MLP+RPR$_{\tau}$    0.385619  0.012184    15   
                           MLP+L2              0.385462  0.014493    15   
...                                                 ...       ...   ...   
acc_odds     German Credit MLP+RPR$_{\tau}$    0.646668  0.074699    14   
                           MLP+RPR$_{\rho_s}$  0.646608  0.063586    16   
                           MLP+RPR$_{\rho}$    0.641057  0.038716    16   
                           MLP+RPR$_{\xi}$     0.640351  0.062612    15   
                           MLP                 0.619366  0.070376    30   

                                              Performance            Fairness  \
                                                     mean       std      mean   
fitness_rule dataset       method                                               
mcc_parity   Adult Income  MLP+RPR$_{\rho}$      0.580085  0.010404  0.186951   
                           MLP+RPR$_{\xi}$       0.578472  0.010924  0.190892   
                           MLP                   0.576385  0.009136  0.190730   
                           MLP+RPR$_{\tau}$      0.576804  0.008372  0.191185   
                           MLP+L2                0.575642  0.010278  0.190180   
...                                                   ...       ...       ...   
acc_odds     German Credit MLP+RPR$_{\tau}$      0.747857  0.033668  0.101189   
                           MLP+RPR$_{\rho_s}$    0.742500  0.028107  0.095892   
                           MLP+RPR$_{\rho}$      0.734375  0.028976  0.093318   
                           MLP+RPR$_{\xi}$       0.747667  0.020430  0.107316   
                           MLP                   0.740000  0.031786  0.120634   

                                                         formatted_fitness  \
                                                    std                      
fitness_rule dataset       method                                            
mcc_parity   Adult Income  MLP+RPR$_{\rho}$    0.013465  $0.393 (\pm0.01)$   
                           MLP+RPR$_{\xi}$     0.012732  $0.388 (\pm0.01)$   
                           MLP                 0.008002  $0.386 (\pm0.01)$   
                           MLP+RPR$_{\tau}$    0.010923  $0.386 (\pm0.01)$   
                           MLP+L2              0.011652  $0.385 (\pm0.01)$   
...                                                 ...                ...   
acc_odds     German Credit MLP+RPR$_{\tau}$    0.067014  $0.647 (\pm0.07)$   
                           MLP+RPR$_{\rho_s}$  0.049303  $0.647 (\pm0.06)$   
                           MLP+RPR$_{\rho}$    0.044720  $0.641 (\pm0.04)$   
                           MLP+RPR$_{\xi}$     0.058156  $0.640 (\pm0.06)$   
                           MLP                 0.068075  $0.619 (\pm0.07)$   

                                              formatted_performance  \
                                                                      
fitness_rule dataset       method                                     
mcc_parity   Adult Income  MLP+RPR$_{\rho}$       $0.580 (\pm0.01)$   
                           MLP+RPR$_{\xi}$        $0.578 (\pm0.01)$   
                           MLP                    $0.576 (\pm0.01)$   
                           MLP+RPR$_{\tau}$       $0.577 (\pm0.01)$   
                           MLP+L2                 $0.576 (\pm0.01)$   
...                                                             ...   
acc_odds     German Credit MLP+RPR$_{\tau}$       $0.748 (\pm0.03)$   
                           MLP+RPR$_{\rho_s}$     $0.742 (\

In [70]:
selected_columns = ['formatted_fitness', 'formatted_performance', 'formatted_fairness']
for fitness_rule in fitness_rules:
    grouped_results.loc[fitness_rule][selected_columns].to_latex(f'tables/grouped_results_{fitness_rule}_crp.tex')
     #.to_latex(f'tables/grouped_results_{fitness_rule}_crp.tex', columns=selected_columns))

In [72]:
significances = significance[significance.method == r'MLP+RPR$_{\xi}$'][[r'MLP', r'MLP+L2', r'MLP+RPR$_{\rho}$', r'MLP+RPR$_{\rho_s}$', r'MLP+RPR$_{\tau}$']]
significances.to_latex(f'tables/aso_rpr_variations.tex')
significances

MLP    MLP+L2  \
fitness_rule            dataset                                 
Max(MCC - Stat. Parity) Adult Income       0.801043  0.678098   
                        Bank Marketing     1.000000  0.461020   
                        Compas Recidivism  0.015765  0.172591   
                        German Credit      1.000000  1.000000   
Max(MCC - Eq. Opp.)     Adult Income       0.085117  0.030187   
                        Bank Marketing     1.000000  1.000000   
                        Compas Recidivism  0.190255  0.230990   
                        German Credit      1.000000  0.707371   
Max(MCC - Eq. Odds)     Adult Income       0.289463  0.238521   
                        Bank Marketing     1.000000  1.000000   
                        Compas Recidivism  0.012811  0.038406   
                        German Credit      1.000000  0.629252   
Max(Acc - Stat. Parity) Adult Income       0.764407  0.357878   
                        Bank Marketing     1.000000  1.000000   
                        Compas Recidivism  0.419125  0.597477   
                        German Credit      0.920390  1.000000   
Max(Acc - Eq. Opp.)     Adult Income       0.351249  0.340874   
                        Bank Marketing     1.000000  0.562121   
                        Compas Recidivism  0.019508  0.037430   
                        German Credit      0.646643  1.000000   
Max(Acc - Eq. Odds)     Adult Income       1.000000  0.738295   
                        Bank Marketing     0.188616  0.154233   
                        Compas Recidivism  0.085419  0.410456   
                        German Credit      0.256817  0.111484   

                                           MLP+RPR$_{\rho}$  \
fitness_rule            dataset                               
Max(MCC - Stat. Parity) Adult Income               1.000000   
                        Bank Marketing             0.799061   
                        Compas Recidivism          0.184003   
                        German Credit              1.000000   
Max(MCC - Eq. Opp.)     Adult Income               0.256104   
                        Bank Marketing             0.791482   
                        Compas Recidivism          0.600127   
                        German Credit              1.000000   
Max(MCC - Eq. Odds)     Adult Income               0.246740   
                        Bank Marketing             1.000000   
                        Compas Recidivism          0.159966   
                        German Credit              0.375569   
Max(Acc - Stat. Parity) Adult Income               0.306553   
                        Bank Marketing             1.000000   
                        Compas Recidivism          0.477280   
                        German Credit              1.000000   
Max(Acc - Eq. Opp.)     Adult Income               0.263599   
                        Bank Marketing             0.837827   
                        Compas Recidivism          0.055873   
                        German Credit              1.000000   
Max(Acc - Eq. Odds)     Adult Income               0.676013   
                        Bank Marketing             0.210039   
                        Compas Recidivism          0.257661   
                        German Credit              0.501892   

                                           MLP+RPR$_{\rho_s}$  \
fitness_rule            dataset                                 
Max(MCC - Stat. Parity) Adult Income                 0.614700   
                        Bank Marketing               0.735043   
                        Compas Recidivism            0.146900   
                        German Credit                0.244125   
Max(MCC - Eq. Opp.)     Adult Income                 0.458334   
                        Bank Marketing               1.000000   
                        Compas Recidivism            0.334401   
                        German Credit                0.889051   
Max(MCC - Eq. Odds)     Adult Income                 0.627617   
   